In [3]:
import os
import sys
import time
import torch
import shutil
import tarfile
import speedtest
import torchvision
import numpy as np
import urllib.request
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as func
import torch.nn.functional as tfunc
from IPython.display import display
import torch.backends.cudnn as cudnn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from sklearn.metrics._ranking import roc_auc_score
from torch.optim.lr_scheduler import ReduceLROnPlateau


%matplotlib inline

In [4]:
class DATASET(Dataset):
    def __init__(self , transform=None , Train=False ) -> None:
        
        self.records = []
        self.transform = transform
        
        if not Train:
            path = "dataset/test.txt"
        else:
            path = "TO BE DEFINED"
        
        with open(path , "r") as file:
            # print(len(file.readlines()))
            for i in file.readlines():
                record = i.split()
                item = (record[0] , list(map(int , record[1:])))
                self.records.append(item)
                # print(os.path.exists(f"database/{record[0]}"))
                
                
        
        print("Total number of Imgs" , len(self.records))
    
    def __getitem__(self, index) :
        
        IMG = self.records[index]
        # print(IMG)
        PIL_img = Image.open("database/" +IMG[0]).convert("RGB")
        LABEL_img = torch.FloatTensor( IMG[1])
        if self.transform != None: PIL_img = self.transform(PIL_img)
        
        return PIL_img , LABEL_img
    
    def __len__(self):
        return len(self.records)
        
        
                
        
        

In [5]:
CKPT_PATH = 'models/model.pth.tar'
N_CLASSES = 14
CLASS_NAMES = [ 'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia',
                'Pneumothorax', 'Consolidation', 'Edema', 'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia']
DATA_DIR = './ChestX-ray14/images'
TEST_IMAGE_LIST = './ChestX-ray14/labels/test_list.txt'
BATCH_SIZE = 64

In [6]:
class DenseNet121(nn.Module):
    def __init__(self, out_size):
        super(DenseNet121, self).__init__()
        self.densenet121 = torchvision.models.densenet121(pretrained=True)
        num_ftrs = self.densenet121.classifier.in_features
        self.densenet121.classifier = nn.Sequential(
            nn.Linear(num_ftrs, out_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.densenet121(x)
        return x

In [7]:
def compute_AUCs(gt, pred):
    """Computes Area Under the Curve (AUC) from prediction scores.
    Args:
        gt: Pytorch tensor on GPU, shape = [n_samples, n_classes]
          true binary labels.
        pred: Pytorch tensor on GPU, shape = [n_samples, n_classes]
          can either be probability estimates of the positive class,
          confidence values, or binary decisions.
    Returns:
        List of AUROCs of all classes.
    """
    AUROCs = []
    gt_np = gt.cpu().numpy()
    pred_np = pred.cpu().numpy()
    for i in range(N_CLASSES):
        AUROCs.append(roc_auc_score(gt_np[:, i], pred_np[:, i]))
    return AUROCs

In [10]:

def main():

    cudnn.benchmark = True

    # initialize and load the model
    device_ids = [0, 1, 2]
    model = DenseNet121(N_CLASSES).cuda()
    model = torch.nn.DataParallel(model , device_ids=device_ids).cuda()

    if os.path.isfile(CKPT_PATH):
        print("=> loading checkpoint")
        checkpoint = torch.load(CKPT_PATH)
        model.load_state_dict(checkpoint['state_dict'] , strict=False)
        print("=> loaded checkpoint")
    else:
        print("=> no checkpoint found")

    normalize = transforms.Normalize([0.485, 0.456, 0.406],
                                     [0.229, 0.224, 0.225])

    test_dataset = DATASET(transform=transforms.Compose([
                                        transforms.Resize(256),
                                        transforms.TenCrop(224),
                                        transforms.Lambda
                                        (lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
                                        transforms.Lambda
                                        (lambda crops: torch.stack([normalize(crop) for crop in crops]))
                                    ]))
    test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE,
                             shuffle=False, num_workers=8, pin_memory=True)

    # initialize the ground truth and output tensor
    gt = torch.FloatTensor()
    gt = gt.cuda()
    pred = torch.FloatTensor()
    pred = pred.cuda()

    # switch to evaluate mode
    model.eval()

    for i, (inp, target) in enumerate(test_loader):
        target = target.cuda()
        gt = torch.cat((gt, target), 0)
        bs, n_crops, c, h, w = inp.size()
        input_var = torch.autograd.Variable(inp.view(-1, c, h, w).cuda(), volatile=True)
        output = model(input_var)
        output_mean = output.view(bs, n_crops, -1).mean(1)
        pred = torch.cat((pred, output_mean.data), 0)

    AUROCs = compute_AUCs(gt, pred)
    AUROC_avg = np.array(AUROCs).mean()
    print('The average AUROC is {AUROC_avg:.3f}'.format(AUROC_avg=AUROC_avg))
    for i in range(N_CLASSES):
        print('The AUROC of {} is {}'.format(CLASS_NAMES[i], AUROCs[i]))

In [11]:
main()

/home/kmit-nvidia/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kmit-nvidia/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


=> loading checkpoint
=> loaded checkpoint
Total number of Imgs 22433


/tmp/ipykernel_906272/1635720245.py:45: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  input_var = torch.autograd.Variable(inp.view(-1, c, h, w).cuda(), volatile=True)


The average AUROC is 0.568
The AUROC of Atelectasis is 0.5078725068374977
The AUROC of Cardiomegaly is 0.5815592907352373
The AUROC of Effusion is 0.5700464864857285
The AUROC of Infiltration is 0.5726122360771466
The AUROC of Mass is 0.5268263863853909
The AUROC of Nodule is 0.5597481061271761
The AUROC of Pneumonia is 0.5268196547554272
The AUROC of Pneumothorax is 0.5709158377078678
The AUROC of Consolidation is 0.6265536285261593
The AUROC of Edema is 0.662728633225793
The AUROC of Emphysema is 0.5208553104867718
The AUROC of Fibrosis is 0.5778900765009758
The AUROC of Pleural_Thickening is 0.5469677528805368
The AUROC of Hernia is 0.6013906522816352
